In [ ]:
 import numpy as np
import pandas as pd

#Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

#Text Handling Libraries
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

df=pd.read_csv('/content/drive/MyDrive/Data/BigBasket Products.csv',index_col='index')
df.head(10)

,product,category,sub_category,brand,sale_price,market_price,type,rating,description
index,,,,,,,,,
1,Garlic Oil - Vegetarian Capsule 500 mg,Beauty & Hygiene,Hair Care,Sri Sri Ayurveda,220.0,220.0,Hair Oil & Serum,4.1,This Product contains Garlic Oil that is known...
2,Water Bottle - Orange,"Kitchen, Garden & Pets",Storage & Accessories,Mastercook,180.0,180.0,Water & Fridge Bottles,2.3,"Each product is microwave safe (without lid), ..."
3,"Brass Angle Deep - Plain, No.2",Cleaning & Household,Pooja Needs,Trm,119.0,250.0,Lamp & Lamp Oil,3.4,"A perfect gift for all occasions, be it your m..."
4,Cereal Flip Lid Container/Storage Jar - Assort...,Cleaning & Household,Bins & Bathroom Ware,Nakoda,149.0,176.0,"Laundry, Storage Baskets",3.7,Multipurpose container with an attractive desi...
5,Creme Soft Soap - For Hands & Body,Beauty & Hygiene,Bath & Hand Wash,Nivea,162.0,162.0,Bathing Bars & Soaps,4.4,Nivea Creme Soft Soap gives your skin the best...
6,Germ - Removal Multipurpose Wipes,Cleaning & Household,All Purpose Cleaners,Nature Protect,169.0,199.0,Disinfectant Spray & Cleaners,3.3,Stay protected from contamination with Multipu...
7,Multani Mati,Beauty & Hygiene,Skin Care,Satinance,58.0,58.0,Face Care,3.6,Satinance multani matti is an excellent skin t...
8,Hand Sanitizer - 70% Alcohol Base,Beauty & Hygiene,Bath & Hand Wash,Bionova,250.0,250.0,Hand Wash & Sanitizers,4.0,70%Alcohol based is gentle of hand leaves skin...
9,Biotin & Collagen Volumizing Hair Shampoo + Bi...,Beauty & Hygiene,Hair Care,StBotanica,1098.0,1098.0,Shampoo & Conditioner,3.5,"An exclusive blend with Vitamin B7 Biotin, Hyd..."


In [ ]:
print(df.shape)
df.isna().sum()


(27555, 9)


product            1
category           0
sub_category       0
brand              1
sale_price         0
market_price       0
type               0
rating          8626
description      115
dtype: int64

In [ ]:
df.dropna(inplace=True)
print(df.shape)

(18840, 9)


In [ ]:
df['category'].value_counts()

Beauty & Hygiene            5460
Kitchen, Garden & Pets      2494
Snacks & Branded Foods      2468
Gourmet & World Food        2364
Foodgrains, Oil & Masala    2173
Cleaning & Household        2091
Bakery, Cakes & Dairy        665
Beverages                    630
Baby Care                    495
Name: category, dtype: int64

In [ ]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['description'])
tfidf_matrix.shape

(18840, 23342)

In [ ]:
len(tfidf.get_feature_names())

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


23342

In [ ]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim

array([[1.        , 0.01632718, 0.00999603, ..., 0.01056047, 0.01133156,
        0.        ],
       [0.01632718, 1.        , 0.00719713, ..., 0.        , 0.        ,
        0.        ],
       [0.00999603, 0.00719713, 1.        , ..., 0.00635776, 0.        ,
        0.        ],
       ...,
       [0.01056047, 0.        , 0.00635776, ..., 1.        , 0.        ,
        0.        ],
       [0.01133156, 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [ ]:
indices=pd.Series(df.index,index=df['product']).drop_duplicates()

In [ ]:
def get_similar(title,cosine_sim):
  idx=indices[title]
  sim_score=list(enumerate(cosine_sim[idx]))
  sim_score.sort(reverse=True,key=lambda x:x[1])
  sim_score=sim_score[1:11]
  indexes=[i[0] for i in sim_score]
  items=df['product'].iloc[indexes]
  return items
get_similar('Hand Sanitizer - 70% Alcohol Base',cosine_sim)

index
11582                   Biotin & Collagen Hair Conditioner
22148            Biotin & Collagen Strenghtening Hair Mask
18491                      Conditioner - Biotin & Collagen
5251     Bounce Hair Recovery Complex, Biotin & Seleniu...
2709     Pro Keratin & Argan Oil Smooth Therapy Conditi...
18867    Red Onion Hair Shampoo - With Biotin, Argan Oi...
23324                                   Protein Hair Cream
16492    Moroccan Argan Hair Oil - Jojoba, Almond, Cast...
5457     Hair Strengthening Conditioner - For Thicker &...
6720     Hair Strengthening Conditioner - For Thicker &...
Name: product, dtype: object

In [ ]:
df2=df.copy()

In [ ]:
for i in ['category','sub_category','type']:
  df2[i]=df2[i].apply(lambda x: re.split('& |, |\*|\n',str(x).strip()))

In [ ]:
df2.head()

,product,category,sub_category,brand,sale_price,market_price,type,rating,description
index,,,,,,,,,
1,Garlic Oil - Vegetarian Capsule 500 mg,"[Beauty , Hygiene]",[Hair Care],Sri Sri Ayurveda,220.0,220.0,"[Hair Oil , Serum]",4.1,This Product contains Garlic Oil that is known...
2,Water Bottle - Orange,"[Kitchen, Garden , Pets]","[Storage , Accessories]",Mastercook,180.0,180.0,"[Water , Fridge Bottles]",2.3,"Each product is microwave safe (without lid), ..."
3,"Brass Angle Deep - Plain, No.2","[Cleaning , Household]",[Pooja Needs],Trm,119.0,250.0,"[Lamp , Lamp Oil]",3.4,"A perfect gift for all occasions, be it your m..."
4,Cereal Flip Lid Container/Storage Jar - Assort...,"[Cleaning , Household]","[Bins , Bathroom Ware]",Nakoda,149.0,176.0,"[Laundry, Storage Baskets]",3.7,Multipurpose container with an attractive desi...
5,Creme Soft Soap - For Hands & Body,"[Beauty , Hygiene]","[Bath , Hand Wash]",Nivea,162.0,162.0,"[Bathing Bars , Soaps]",4.4,Nivea Creme Soft Soap gives your skin the best...


In [ ]:
df2['type']=df2['type'].apply(lambda x: [i.replace(" ","").lower() for i in x])
df2['category']=df2['category'].apply(lambda x: [i.replace(" ","").lower() for i in x])
df2['sub_category']=df2['sub_category'].apply(lambda x: [i.replace(" ","").lower() for i in x])
df2['brand']=df2['brand'].apply(lambda x: str(x).lower())


In [ ]:
def merged_value(x):
  return ' '.join(x['category']) + ' ' + ' '.join(x['sub_category']) + ' '+x['brand']+' ' +' '.join( x['type'])
df2['merged']=df2.apply(merged_value,axis=1)
df2.head()

,product,category,sub_category,brand,sale_price,market_price,type,rating,description,merged
index,,,,,,,,,,
1,Garlic Oil - Vegetarian Capsule 500 mg,"[beauty, hygiene]",[haircare],sri sri ayurveda,220.0,220.0,"[hairoil, serum]",4.1,This Product contains Garlic Oil that is known...,beauty hygiene haircare sri sri ayurveda hair...
2,Water Bottle - Orange,"[kitchen, garden, pets]","[storage, accessories]",mastercook,180.0,180.0,"[water, fridgebottles]",2.3,"Each product is microwave safe (without lid), ...",kitchen garden pets storage accessories master...
3,"Brass Angle Deep - Plain, No.2","[cleaning, household]",[poojaneeds],trm,119.0,250.0,"[lamp, lampoil]",3.4,"A perfect gift for all occasions, be it your m...",cleaning household poojaneeds trm lamp lampoil
4,Cereal Flip Lid Container/Storage Jar - Assort...,"[cleaning, household]","[bins, bathroomware]",nakoda,149.0,176.0,"[laundry, storagebaskets]",3.7,Multipurpose container with an attractive desi...,cleaning household bins bathroomware nakoda la...
5,Creme Soft Soap - For Hands & Body,"[beauty, hygiene]","[bath, handwash]",nivea,162.0,162.0,"[bathingbars, soaps]",4.4,Nivea Creme Soft Soap gives your skin the best...,beauty hygiene bath handwash nivea bathingbars...


In [ ]:
tfidf = CountVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df2['merged'])
tfidf_matrix.shape

(18840, 2672)

In [ ]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim

array([[10.,  0.,  0., ...,  0.,  0.,  2.],
       [ 0.,  8.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  6., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  8.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  4.,  0.],
       [ 2.,  0.,  0., ...,  0.,  0., 11.]])

In [ ]:
indices=pd.Series(df2.index,index=df2['product']).drop_duplicates()

In [ ]:
def get_similar(title,cosine_sim):
  idx=indices[title]
  sim_score=list(enumerate(cosine_sim[idx]))
  sim_score.sort(reverse=True,key=lambda x:x[1])
  sim_score=sim_score[1:11]
  indexes=[i[0] for i in sim_score]
  items=df2['product'].iloc[indexes]
  return items
get_similar('Cadbury Perk - Chocolate Bar',cosine_sim)

index
4955                                   Pickle - Gooseberry
21303                             Pickle - Mixed Vegetable
26457                                      Pickle - Garlic
367                     Andhra Special Allam/Ginger Pickle
662                                           Dates Pickle
1260     Homemade  Seedless Amla Pickle - No Preservatives
1398                                       Thokku - Tomato
1931                            Achar - Sweet & Sour Berry
2080                           Pickle - Red Stuffed Chilli
2470                           Pickle - Punjabi Pacharanga
Name: product, dtype: object